# 🔧 Model Configuration Manager for RAG Analysis

This notebook allows you to create, manage, and select between different **LLM model configurations** for use with tools like `ChatGPTAnalyzer`.

Each configuration includes:
- 🤖 **Model name** (e.g., `gpt-4o`, `mistral-7b-instruct`, `llama3-70b-instruct`)
- 🌐 **Base URL** (e.g., `https://api.openai.com/v1`, `http://localhost:8080/v1`)
- 🔐 **API Key** (e.g., `sk-...` or `ollama`, `sk-noauth`)

---

## 🗃️ Configuration Storage

All configurations are stored in the following JSON file:

```
~/.secrets/model_configs.json
```

This file will look something like:

```json
{
  "openai": {
    "model": "gpt-4o",
    "base_url": "https://api.openai.com/v1",
    "api_key": "sk-..."
  },
  "mistral": {
    "model": "mistral-7b-instruct",
    "base_url": "http://localhost:8080/v1",
    "api_key": "sk-noauth"
  },
  "_default": "openai"
}
```

You can **manually edit or copy** this file if needed to:
- Share configurations with others
- Maintain consistent setups across environments
- Set or change the default model (`"_default"` key)

---

## ✅ How to Use This Notebook

1. Run the notebook cells to:
   - Add or update a named model configuration
   - Optionally set it as the default

2. Use your configurations in tools like `ChatGPTAnalyzer`:

```python
from Open_AI_RAG_manager import ChatGPTAnalyzer

# Use specific model
analyzer = ChatGPTAnalyzer.from_config("mistral", yaml_content=my_yaml)

# Or just use the default model
analyzer = ChatGPTAnalyzer.from_config(yaml_content=my_yaml)
```

---

> 💡 Tip: You can define as many configurations as you like — great for testing different LLMs or backends like OpenAI, LM Studio, Ollama, and Hugging Face APIs.



In [ ]:
import json
from pathlib import Path
from getpass import getpass
from IPython.display import display, Markdown
from ipywidgets import widgets, Button, VBox, HBox, Output

CONFIG_FILE = Path.home() / ".secrets" / "model_configs.json"
CONFIG_FILE.parent.mkdir(exist_ok=True)

def load_configs():
    if CONFIG_FILE.exists():
        with CONFIG_FILE.open() as f:
            return json.load(f)
    return {}

def save_configs(configs):
    with CONFIG_FILE.open("w") as f:
        json.dump(configs, f, indent=2)

def list_configs():
    configs = load_configs()
    if not configs:
        display(Markdown("❌ **No configurations found.**"))
        return

    default_name = configs.get("_default", "None")
    display(Markdown(f"### 📦 Saved Configurations (default: `{default_name}`)"))
    for name, config in configs.items():
        if name == "_default":
            continue
        display(Markdown(f"- `{name}` → model: `{config['model']}`, url: `{config['base_url']}`"))

# Display initial configs
list_configs()

# Create widgets for configuration input
config_name_input = widgets.Text(
    placeholder='e.g., mistral',
    description='🔖 Config Name:',
    style={'description_width': '150px'}
)

model_name_input = widgets.Text(
    placeholder='e.g., mistral-7b-instruct',
    description='🤖 Model Name:',
    style={'description_width': '150px'}
)

base_url_input = widgets.Text(
    placeholder='e.g., http://localhost:8080/v1',
    description='🌐 Base URL:',
    style={'description_width': '150px'}
)

api_key_input = widgets.Password(
    placeholder='e.g., sk-noauth',
    description='🔐 API Key:',
    style={'description_width': '150px'}
)

make_default_checkbox = widgets.Checkbox(
    value=False,
    description='⭐ Set as default?'
)

# Output area for messages
output_area = Output()

# Save button
save_button = Button(
    description='💾 Save Configuration',
    button_style='success',
    tooltip='Save the configuration'
)

def on_save_clicked(b):
    with output_area:
        output_area.clear_output()
        
        # Validate inputs
        name = config_name_input.value.strip()
        model = model_name_input.value.strip()
        base_url = base_url_input.value.strip()
        api_key = api_key_input.value.strip()
        make_default = make_default_checkbox.value
        
        if not name or not model or not base_url or not api_key:
            display(Markdown("❌ **All fields are required!**"))
            return
        
        # Save configuration
        configs = load_configs()
        configs[name] = {
            "model": model,
            "base_url": base_url,
            "api_key": api_key
        }
        
        if make_default:
            configs["_default"] = name
        
        save_configs(configs)
        display(Markdown(f"✅ **Configuration `{name}` saved.**"))
        if make_default:
            display(Markdown(f"🌟 **Set as default configuration.**"))
        
        # Clear inputs
        config_name_input.value = ''
        model_name_input.value = ''
        base_url_input.value = ''
        api_key_input.value = ''
        make_default_checkbox.value = False
        
        # Refresh config list
        display(Markdown("---"))
        list_configs()

save_button.on_click(on_save_clicked)

# Layout
form_box = VBox([
    config_name_input,
    model_name_input,
    base_url_input,
    api_key_input,
    make_default_checkbox,
    save_button
])

# Display form and output
display(Markdown("### 🔧 Add or Update Configuration"))
display(form_box)
display(output_area)

❌ **No configurations found.**

In [ ]:
s